In [1]:
import numpy as mp
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import socket
import import_ipynb
import Model_Owner as mu
import Client as Cli
from Pyfhel import Pyfhel
from Crypto.Cipher import AES #pip install pycryptodome
from base64 import b64encode, b64decode
import os

importing Jupyter notebook from Model_Owner.ipynb
importing Jupyter notebook from Client.ipynb


In [2]:
OWNERS = 1 #Number of Model Owners
TRAINING_DATASET = 'iris.csv'
DELIMITER = ','
SAVEFILE = 'iris_data.sav'
PYFHEL_OBJECTS = []
USERS = []

BLOCK_SIZE_AES = 16

In [3]:
#generate public key generation parameter
GEN_KEY_PARAM = { "p":257,      "r":1,
                            "d":1,        "c":2,
                            "sec":80,     "w":64,
                            "L":10,       "m":-1,
                            "R":3,        "s":0,
                            "gens":[],    "ords":[]}
print ("\nThe Key Generation Parameters are\n")
print (GEN_KEY_PARAM)



The Key Generation Parameters are

{'p': 257, 'r': 1, 'd': 1, 'c': 2, 'sec': 80, 'w': 64, 'L': 10, 'm': -1, 'R': 3, 's': 0, 'gens': [], 'ords': []}


In [4]:
print ("\nGenerating Pyfhel Objects and public Parameters ")

class Algorithm():
    
    def __init__(self):
        self.list_dec_tree = []
        self.client = Cli.Client_Req()
        
    def train_dec_models(self):
        print ("Training Models")
        for i in range(OWNERS):
            model = mu.User(TRAINING_DATASET,DELIMITER,SAVEFILE)
            model.run_model()
            self.client_features = model.test_x
            #model.visualize()
            model.save_model()
            self.list_dec_tree.append(model)
            print ("Model "+ str(i+1) + " trained")
        print ("Models Trained by Users")
            
    def generate_tree_keys(self):
        print ("Generating Tree Keys")
        for i in range(OWNERS):
            self.list_dec_tree[i].aes_tree(BLOCK_SIZE_AES) #create aes key
            self.list_dec_tree[i].Pyfhel_Object() #generate pyfhel object
            self.list_dec_tree[i].public_parameters(GEN_KEY_PARAM) #generate public parameter
            print ("Generating pair of keys for the model owner " + str(i+1)) #for each user generate set of keys
            self.list_dec_tree[i].key_generation()
            self.list_dec_tree[i].eval_key() #generate evaluation helper key for model owners
        
        print ("Tree Key Generation Completed")
    
    def client_key_generation(self):
        print ("Generating Client Keys")
        self.client.get_feature_vector(self.client_features)
        self.client.aes_client(BLOCK_SIZE_AES)  #create aes key
        self.client.Client_Pyfhel() #generate pyfhel object
        self.client.Client_public_parameters(GEN_KEY_PARAM) #generate public parameter
        self.client.Client_key_generation() #for client generate keys
        print ("Client Key Generation Completed")
        
    
    


Generating Pyfhel Objects and public Parameters 


In [5]:
if __name__ == "__main__":
    result = Algorithm()
    result.train_dec_models()
    result.generate_tree_keys()
    result.client_key_generation()
    
    

Training Models
Model 1 trained
Model 2 trained
Model 3 trained
Models Trained by Users
Generating Tree Keys
Generating pair of keys for the model owner 1
Generating pair of keys for the model owner 2
Generating pair of keys for the model owner 3
Tree Key Generation Completed
Generating Client Keys
Client Key Generation Completed
